In [1]:
#%pip install --user cuml-cu11 cudf-cu11 --extra-index-url=https://pypi.nvidia.com

In [2]:
#%load_ext cuml.accel
import cuml
import cupy as cp
import cudf
from cuml.cluster import HDBSCAN, DBSCAN
from cuml.metrics import pairwise_distances
from cuml.manifold import UMAP
from pathlib import Path
import matplotlib.pyplot as plt

# Конфигурация путей
EMBEDDINGS_DIR = Path("embeddings")
CLUSTER_RESULTS_DIR = Path("cluster_results")
CLUSTER_CENTERS_DIR = Path("cluster_centers")
import numpy as np
import pandas as pd
# Параметры кластеризации
CLUSTER_PARAMS = {
    'hdbscan': {
        'min_cluster_size': 150,
        'cluster_selection_epsilon': 0.3,
        'prediction_data': True
    },
    'dbscan': {
        'eps': 0.5,
        'min_samples': 10,
        'metric': 'cosine'
    }
}

# Создание директорий
for dir in [CLUSTER_RESULTS_DIR, CLUSTER_CENTERS_DIR]:
    dir.mkdir(exist_ok=True)

# Загрузка и преобразование данных
processed_df = cudf.read_csv("processed_errors_clean.csv")

def load_gpu_embeddings(embedding_name: str):
    """Загрузка эмбеддингов в GPU память"""
    emb_path = EMBEDDINGS_DIR / f"{embedding_name}.csv"
    return cudf.read_csv(emb_path).values

def get_central_samples_gpu(X, labels, cluster_id, n_samples=10):
    """Поиск центральных образцов на GPU"""
    cluster_mask = labels == cluster_id
    cluster_data = X[cluster_mask]
    
    # Вычисление центра кластера
    center = cluster_data.mean(axis=0)
    
    # Расстояния до центра на GPU
    distances = pairwise_distances(cluster_data, center.reshape(1, -1), 
                metric='cosine').flatten()
    
    # Используем cupy для работы с индексами
    closest_indices = cp.argsort(distances)[:n_samples]
    return closest_indices.get()

def cluster_and_save_gpu(embedding_name, algorithm='hdbscan'):
    """Основная функция кластеризации с GPU ускорением"""
    # Загрузка эмбеддингов в GPU память
    X = load_gpu_embeddings(embedding_name)
    
    # Инициализация кластеризатора
    if algorithm == 'hdbscan':
        clusterer = HDBSCAN(**CLUSTER_PARAMS['hdbscan'])
    elif algorithm == 'dbscan':
        clusterer = DBSCAN(**CLUSTER_PARAMS['dbscan'])
    
    # Кластеризация и конвертация в CPU-массив
    labels = clusterer.fit_predict(X)
    labels_cpu = labels.get()  # Конвертируем метки в CPU

    # Фильтрация шума
    valid_mask = labels_cpu != -1
    filtered_labels = labels_cpu[valid_mask]
    filtered_ids = processed_df.index[valid_mask].to_pandas().values
    
    # Сохранение результатов кластеризации
    result_df = cudf.DataFrame({
        'log_id': filtered_ids,
        'cluster': filtered_labels,
        'error_text': processed_df.loc[valid_mask, 'error_text'].to_pandas().values
    })
    result_path = CLUSTER_RESULTS_DIR / f"{embedding_name}_{algorithm}_clusters.csv"
    result_df.to_csv(result_path, index=False)
    
    # Поиск центральных объектов
    clusters = np.unique(filtered_labels)
    centers_data = []
    
    for cluster_id in clusters:
        sample_indices = get_central_samples_gpu(
            X[valid_mask], 
            filtered_labels, 
            cluster_id
        )
        
        for idx in sample_indices:
            # Явное преобразование типов и обработка NaN
            error_text = str(processed_df.iloc[idx]['error_text'])
            error_text = error_text if not pd.isna(error_text) else ""
            
            centers_data.append({
                'cluster': int(cluster_id),
                'log_id': int(filtered_ids[idx]),
                'error_text': error_text
            })
    
    # Создание и сохранение центров кластеров
    centers_df = cudf.DataFrame(centers_data).astype({
        'cluster': 'int32',
        'log_id': 'int64',
        'error_text': 'str'
    })
    centers_path = CLUSTER_CENTERS_DIR / f"{embedding_name}_{algorithm}_centers.csv"
    centers_df.to_csv(centers_path, index=False)
    
    # Визуализация распределения кластеров
    plot_cluster_distribution(filtered_labels, embedding_name, algorithm)
    
    return result_df, centers_df

def plot_cluster_distribution(labels, embedding_name, algorithm):
    """Визуализация распределения кластеров"""
    sizes = np.bincount(labels[labels != -1])
    plt.figure(figsize=(12,6))
    plt.bar(range(len(sizes)), sizes)
    plt.title(f"Cluster Distribution: {embedding_name} ({algorithm})")
    plt.xlabel("Cluster ID")
    plt.ylabel("Number of Samples")
    plt.savefig(CLUSTER_RESULTS_DIR / f"{embedding_name}_{algorithm}_distribution.png")
    plt.close()


In [3]:
embedding_name = "longformer"
result_df, centers_df = cluster_and_save_gpu(embedding_name, algorithm='hdbscan')
    
print("Результаты сохранены:")
print(f"- Кластеры: {CLUSTER_RESULTS_DIR / f'{embedding_name}_hdbscan_clusters.csv'}")
print(f"- Центры: {CLUSTER_CENTERS_DIR / f'{embedding_name}_hdbscan_centers.csv'}")

Результаты сохранены:
- Кластеры: cluster_results/longformer_hdbscan_clusters.csv
- Центры: cluster_centers/longformer_hdbscan_centers.csv
